In [1]:
## loading essential libraries
import numpy as np, os.path, pandas as pd, sys, matplotlib.pyplot as plt, copy
from pytorch import run_final_model, run_cross_valid, combined_plot, save_outputs, stat_new_data
from pytorch import feature_slist, feature_list, stat, specific, explore, print_optimal_tables, print_stat_tables, combined_plot_noise, learning_curve, plot_learning_curve, load_model
from read_in_out import initiate_data, initiate_RNN_data, analysis_options, ML_analysis, compare_braced_input_data, compare_braced_output_data
from joblib import Parallel, delayed
from pytorch_utilities import transformer
feat_order = ['JA','JM','JRF']#,'MA','MF']
import seaborn as sns

## https://viso.ai/deep-learning/explainable-ai/


In [2]:
### some scripts need to compute
def initiate_naive_braced(dfo, dfn):
    dfo.naive_braced         = copy.deepcopy(dfn.naive)  
    dfo.naive_braced.arg     = copy.deepcopy(dfo.naive.arg)  ## creating an instance of naive_braced
    dfo.naive_braced.arch    = copy.deepcopy(dfo.naive.arch)  ## creating an instance of naive_braced
    dfo.naive_braced.save_name = copy.deepcopy(dfo.naive.save_name)  ## creating an instance of naive_braced
    dfo.naive_braced.subject = 'naive_braced'
    return dfo

def train_final_models(D):
    ## train final model with best-avg-validation accuracy
    for i in range(3):
        specific(D.exposed,i)
        specific(D.naive  ,i)
    return None

def compute_stat(f):
    for D in f:
        for i in range(3):
            D.exposed = stat(D.exposed,i)
            print(D.exposed.subject, 10*'--***--',i)
            D.naive   = stat(D.naive,i)
            print(D.naive.subject, 10*'--***--',i)
            try:
                D.exposed_unseen.subject = 'exposed_unseen'
                D.exposed_unseen = stat(D.exposed_unseen, i)
                print('exposed_unseen', 10*'--***--',i)
            except:
                None
            try:
                D.naive_braced = stat(D.naive_braced, i)
                print('naive_braced', 10*'--***--',i)
            except:
                None
            print(f"\n\n")
    return f

def avg_stat(fm):
    for f in fm:
        for j in [f.exposed, f.naive]:
            a,b = [],[]
            for i in f.feature:
                a.append(j.NRMSE[i])
                b.append(j.pc[i])
            a = pd.concat(a, axis=1)
            b = pd.concat(b, axis=1)
            # print('%',np.around(np.mean(a),2),np.around(np.std(a),2), j.kind, j.subject, 'NRMSE')
            # print('%',np.around(np.mean(b),2),np.around(np.std(b),2), j.kind, j.subject, 'pc')

# avg_stat([fm.LM])
from itertools import chain
def merge_lists(nested_list):
    return list(chain.from_iterable(nested_list))


def plot_final_results(df1, save_name, plot_what):
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']*10  # blue, orange, green
    labels = df1[0].feature_l

    if plot_what == 'pc':
        fig, ax = plt.subplots()
        df = [t.pc for t in df1]
        ax.set_ylabel('pearson correlation', size=12)    
        data = [data_dict[key].values.ravel().tolist() for data_dict in df for key in data_dict.keys()]
        box_width = 0.5
        pos = [i for sl in [np.arange(3)+4.5*i for i in range(len(data)//3)] for i in sl]
        bplot = ax.boxplot(data, positions=pos, patch_artist=True, whis=None, showfliers=False,medianprops=dict(color='black'))    
        for box, color in zip(bplot['boxes'], colors):
            box.set_facecolor(color)
        ax.set_ylim(-1.02,1.02)
        for patch, color in zip(bplot['boxes'], colors):
            patch.set_facecolor(color)
        xtlabels = [t.save_name for t in df1]
        ax.set_xticks(pos[1::3])
        ax.set_xticklabels(xtlabels,rotation=45)
        ax.tick_params(axis='both', labelsize=12)  
        ax.grid(axis='y', color='lightgray', linestyle='-', alpha=0.3)
        from matplotlib.patches import Patch
        legend_patches = [Patch(facecolor=c, label=l) for c, l in zip(colors, labels)]
        legend = ax.legend(handles=legend_patches,ncol=3,bbox_to_anchor=(1.01, 1.09), loc = 'upper right',  columnspacing=1.5)  # You can change the location as needed
        legend.get_frame().set_alpha(1)     
        plt.tight_layout()
        plt.savefig(f'./plots_out/{save_name}.{plot_what}.pdf', dpi=600)
        plt.show()
        plt.close()

    elif plot_what == 'nparams':
        fig, ax = plt.subplots()
        data = merge_lists([t.nparams for t in df1])
        ax.set_ylabel('# parameters', size=12)    
        pos = [i for sl in [np.arange(3)+4.5*i for i in range(len(data)//3)] for i in sl]
        ax.bar(pos, data, color = colors*20)
        ax.set_xlim(pos[0]-0.5, pos[-1]+0.5)
        xtlabels = [t.save_name for t in df1]
        ax.set_xticks(pos[1::3])
        ax.set_xticklabels(xtlabels,rotation=45)
        ax.tick_params(axis='both', labelsize=12)  
        ax.grid(axis='y', color='lightgray', linestyle='-', alpha=0.3)
        from matplotlib.patches import Patch
        legend_patches = [Patch(facecolor=c, label=l) for c, l in zip(colors, labels)]
        legend = ax.legend(handles=legend_patches,ncol=3,bbox_to_anchor=(1.01, 1.09), loc = 'upper right',  columnspacing=1.5)  # You can change the location as needed
        legend.get_frame().set_alpha(1) 
        # plt.tight_layout()
        plt.savefig(f'./plots_out/{save_name}.{plot_what}.pdf', dpi=600)
        plt.show()
        plt.close()

    elif plot_what in ['RMSE', 'NRMSE']:
        if plot_what == 'NRMSE':
            df = [t.NRMSE  for t in df1[1:]]
        elif plot_what == 'RMSE':
            df = [t.RMSE for t in df1[1:]]
        data = [data_dict[key].values.ravel().tolist() for data_dict in df for key in data_dict.keys()]
        box_width = 0.5
        pos = [i for sl in [np.arange(3)+4.5*i for i in range(len(data)//3)] for i in sl]
        xtlabels = [t.save_name for t in df1[1:]]
        for i in np.arange(3):
            fig, ax = plt.subplots()
            ax.set_ylabel(plot_what, size=12)    
            posi    = pos[i::3]
            datai   = data[i::3]
            colorsi = colors[i::3]
            bplot  = ax.boxplot(datai, positions=posi, patch_artist=True, whis=None, showfliers=False,medianprops=dict(color='black'))    
            for box, color in zip(bplot['boxes'], colorsi):
                box.set_facecolor(color)
            for patch, color in zip(bplot['boxes'], colorsi):
                patch.set_facecolor(color)
            # ax.set_ylim(-.2,3.02)
            ax.set_xticks(posi)
            ax.set_xticklabels(xtlabels,rotation=45)
            ax.tick_params(axis='both', labelsize=12)  
            ax.grid(axis='y', color='lightgray', linestyle='-', alpha=0.3)
            from matplotlib.patches import Patch
            legend_patches = [Patch(facecolor=c, label=l) for c, l in zip(colorsi, labels[i:i+1])]
            legend = ax.legend(handles=legend_patches,ncol=1,bbox_to_anchor=(0.5, 1.1), loc = 'upper center',  columnspacing=1.5)  # You can change the location as needed
            legend.get_frame().set_alpha(1) 
            # plt.title()        
            plt.tight_layout()
            plt.savefig(f'./plots_out/{save_name}.{plot_what}.{i}.pdf', dpi=600)
            plt.show()
            plt.close()

In [1]:
### function to load_model_index
def load_model_index(fm, fm_braced, data_kind):
    if 'LM' in data_kind:
        fm.LM.exposed.arg      = [43, 43, 43]
        fm.LM.naive.arg        = [43, 43, 43]
        fm.LM.exposed.arch, fm.LM.naive.arch  = ['LM']*3, ['LM']*3
        fm.LM.exposed_unseen     = copy.deepcopy(fm.LM.exposed)
        fm.LM.save_name, fm.LM.exposed.save_name, fm.LM.naive.save_name, fm.LM.exposed_unseen.save_name = ('LM',)*4
        fm.LM = initiate_naive_braced(fm.LM, fm_braced.LM) ## creating an instance of naive_braced
        
    if 'rf' in data_kind:
        fm.rf.exposed.arg      = [5, 5, 5]
        fm.rf.naive.arg        = [1, 5, 17]
        fm.rf.exposed.arch, fm.rf.naive.arch = ['rf']*3, ['rf']*3
        fm.rf.exposed_unseen     = copy.deepcopy(fm.rf.exposed)
        fm.rf.save_name, fm.rf.exposed.save_name, fm.rf.naive.save_name, fm.rf.exposed_unseen.save_name = ('RF',)*4
        fm.rf = initiate_naive_braced(fm.rf, fm_braced.rf) ## creating an instance of naive_braced

    if 'transformer' in data_kind:
        fm.transformer.exposed.arg      = [23,23,23]
        fm.transformer.naive.arg        = [23,23,23]
        fm.transformer.exposed.arch, fm.transformer.naive.arch = ['transformer']*3, ['transformer']*3
        fm.transformer.exposed_unseen     = copy.deepcopy(fm.transformer.exposed)
        fm.transformer.save_name, fm.transformer.exposed.save_name, fm.transformer.naive.save_name, fm.transformer.exposed_unseen.save_name = ('transformer',)*4
        fm.transformer = initiate_naive_braced(fm.transformer, fm_braced.transformer) ## creating an instance of naive_braced
            
    if 'xgbr' in data_kind:
        fm.xgbr.exposed.arg        = [96, 96, 96]
        fm.xgbr.naive.arg          = [96, 96, 218]
        fm.xgbr.exposed.arch, fm.xgbr.naive.arch = ['xgbr']*3, ['xgbr']*3
        fm.xgbr.exposed_unseen     = copy.deepcopy(fm.xgbr.exposed)
        fm.xgbr.save_name, fm.xgbr.exposed.save_name, fm.xgbr.naive.save_name, fm.xgbr.exposed_unseen.save_name = ('XGBR',)*4
        fm.xgbr = initiate_naive_braced(fm.xgbr, fm_braced.xgbr) ## creating an instance of naive_braced

    if 'NN' in data_kind:
        fm.NN.exposed.arg        = [1043, 1043, 1043]
        fm.NN.naive.arg          = [1043, 1091, 1046] 
        fm.NN.exposed.arch, fm.NN.naive.arch = ['NN']*3, ['NN']*3
        fm.NN.exposed_unseen     = copy.deepcopy(fm.NN.exposed)
        fm.NN.save_name, fm.NN.exposed.save_name, fm.NN.naive.save_name, fm.NN.exposed_unseen.save_name = ('FFNN',)*4
        fm.NN = initiate_naive_braced(fm.NN, fm_braced.NN) ## creating an instance of naive_braced

    if 'CNNLSTM' in data_kind:
        fm.CNNLSTM.exposed.arg        = [387, 387, 23]
        fm.CNNLSTM.naive.arg          = [387, 387, 391] 
        fm.CNNLSTM.exposed.arch, fm.CNNLSTM.naive.arch = ['CNNLSTM']*3, ['CNNLSTM']*3
        fm.CNNLSTM.exposed_unseen     = copy.deepcopy(fm.CNNLSTM.exposed)
        fm.CNNLSTM.save_name, fm.CNNLSTM.exposed.save_name, fm.CNNLSTM.naive.save_name, fm.CNNLSTM.exposed_unseen.save_name = ('CNNLSTM',)*4
        fm.CNNLSTM = initiate_naive_braced(fm.CNNLSTM, fm_braced.CNNLSTM) ## creating an instance of naive_braced
    
    if 'RNN' in data_kind:    
        fm.VRNN = copy.deepcopy(fm.RNN) 
        fm.LSTM = copy.deepcopy(fm.RNN) 
        fm.GRU  = copy.deepcopy(fm.RNN) 
        
        fm.VRNN.exposed.arg       = [3119, 3411, 3125] 
        fm.VRNN.naive.arg         = [3169, 2051, 3411] 
        fm.VRNN.exposed.arch, fm.VRNN.naive.arch = ['RNN']*3, ['RNN']*3   ## (SimpleRNN, LSTM, GRU)
        fm.VRNN.exposed_unseen     = copy.deepcopy(fm.VRNN.exposed)
        fm.VRNN.save_name, fm.VRNN.exposed.save_name, fm.VRNN.naive.save_name, fm.VRNN.exposed_unseen.save_name = ('VRNN',)*4
        fm.VRNN = initiate_naive_braced(fm.VRNN, fm_braced.RNN) ## creating an instance of naive_braced
    
        fm.LSTM.exposed.arg       = [3458,3666,4396] 
        fm.LSTM.naive.arg         = [3458,3736,4396] 
        fm.LSTM.exposed.arch, fm.LSTM.naive.arch = ['RNN']*3, ['RNN']*3 
        fm.LSTM.exposed_unseen     = copy.deepcopy(fm.LSTM.exposed)
        fm.LSTM.save_name, fm.LSTM.exposed.save_name, fm.LSTM.naive.save_name, fm.LSTM.exposed_unseen.save_name = ('LSTM',)*4
        fm.LSTM = initiate_naive_braced(fm.LSTM, fm_braced.RNN) ## creating an instance of naive_braced
    
        fm.GRU.exposed.arg        = [9153, 10029, 10301]
        fm.GRU.naive.arg          = [9845, 8309, 6916 ]
        fm.GRU.exposed.arch, fm.GRU.naive.arch = ['RNN']*3, ['RNN']*3   ## (SimpleRNN, LSTM, GRU)
        fm.GRU.exposed_unseen     = copy.deepcopy(fm.GRU.exposed)
        fm.GRU.save_name, fm.GRU.exposed.save_name, fm.GRU.naive.save_name, fm.GRU.exposed_unseen.save_name = ('GRU',)*4
        fm.GRU = initiate_naive_braced(fm.GRU, fm_braced.RNN) ## creating an instance of naive_braced

    if 'CNN' in data_kind:
        fm.CNN.exposed.arg        = [28, 243, 31]  
        fm.CNN.naive.arg          = [36,  211, 217] 
        fm.CNN.exposed.arch, fm.CNN.naive.arch = ['CNN']*3, ['CNN']*3
        fm.CNN.exposed_unseen     = copy.deepcopy(fm.CNN.exposed)
        fm.CNN.save_name, fm.CNN.exposed.save_name, fm.CNN.naive.save_name, fm.CNN.exposed_unseen.save_name = ('CNN',)*4
        fm.CNN = initiate_naive_braced(fm.CNN, fm_braced.CNN) ## creating an instance of naive_braced
    print(f'Here argument n is n+2th line in the file ... ')

    if 'convLSTM' in data_kind:
        fm.convLSTM.exposed.arg        = [26, 51, 31]  
        fm.convLSTM.naive.arg          = [36,  33, 26] 
        fm.convLSTM.exposed.arch, fm.convLSTM.naive.arch = ['convLSTM']*3, ['convLSTM']*3
        fm.convLSTM.exposed_unseen     = copy.deepcopy(fm.convLSTM.exposed)
        fm.convLSTM.save_name, fm.convLSTM.exposed.save_name, fm.convLSTM.naive.save_name, fm.convLSTM.exposed_unseen.save_name = ('convLSTM',)*4
        fm.convLSTM = initiate_naive_braced(fm.convLSTM, fm_braced.convLSTM) ## creating an instance of naive_braced

    if 'convLSTM' in data_kind:
        fm.convLSTM.exposed.arg        = [218, 243, 406]  
        fm.convLSTM.naive.arg          = [228, 225, 19] 
        fm.convLSTM.exposed.arch, fm.convLSTM.naive.arch = ['convLSTM']*3, ['convLSTM']*3
        fm.convLSTM.exposed_unseen     = copy.deepcopy(fm.convLSTM.exposed)
        fm.convLSTM.save_name, fm.convLSTM.exposed.save_name, fm.convLSTM.naive.save_name, fm.convLSTM.exposed_unseen.save_name = ('convLSTM',)*4
        fm.convLSTM = initiate_naive_braced(fm.convLSTM, fm_braced.convLSTM) ## creating an instance of naive_braced

    return fm

In [4]:
# window=5
# mm = 'ResNet'
# fm = ML_analysis('final_model_list', [f'{mm}'], window)
# fmb = ML_analysis('final_model_list', [f'{mm}'], window, add1 = 'Braced_')
# fm = load_model_index(fm, fmb, [f'{mm}'])
# D = fm.ResNet

# for i in [0,1,2]:
#     specific(D.exposed, i)
#     specific(D.naive, i)

# print('--***--'*15)
# compute_stat([D])

# print(D.exposed.pc['JA'].mean().mean(),        D.exposed.pc['JM'].mean().mean(),        D.exposed.pc['JRF'].mean().mean())
# print(D.naive.pc['JA'].mean().mean(),          D.naive.pc['JM'].mean().mean(),          D.naive.pc['JRF'].mean().mean())
# print(D.exposed_unseen.pc['JA'].mean().mean(), D.exposed_unseen.pc['JM'].mean().mean(), D.exposed_unseen.pc['JRF'].mean().mean())

sys.exit()

SystemExit: 

/Users/kpxp057/anaconda3/envs/gamma/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [24]:
def return_ab(model_name):
    data_kind = [model_name]
    if 'LM' in data_kind:
        a       = [43, 43, 43]
        b       = [43, 43, 43]
        
    if 'rf' in data_kind:
        a     = [6, 12, 6]
        b        = [0, 12, 20]

    if 'transformer' in data_kind:
        a      = [23,23,23]
        b        = [23,23,23]
            
    if 'xgbr' in data_kind:
        a        = [96, 96, 96]
        b         = [96, 46, 218]

    if 'NN' in data_kind:
        a       = [1043, 1043, 81]
        b          = [1043, 1091, 1046] ## 1043, 1091, 1043

    if 'CNNLSTM' in data_kind:
        a        = [387, 387, 23]
        b          = [387, 387, 391] ## 389


    if 'RNN' in data_kind:    
        a       = [3119, 3411, 3125] + [3458,3669,4396] + [9153, 10029, 10301]
        b        = [3169, 2051, 3411] + [3458,3669,4396] + [9845, 8309, 6916 ]

    if 'CNN' in data_kind:
        a        = [28, 243, 31]  
        b          = [36,  211, 217] 

    if 'convLSTM' in data_kind:
        a       = [218, 243, 406]  
        b         = [7,  13, 56]
    f = ['JA','JM','JRF']*3
    
    if model_name in ['rf']:
        a = [f'model_{model_name}_{f[enum]}_exposed.hv_{i}.pkl' for enum,i in  enumerate(a)]
        b = [f'model_{model_name}_{f[enum]}_naive.hv_{i}.pkl' for enum,i in  enumerate(b)]
    elif model_name in ['xgbr']:
        a = [f'model_{model_name}_{f[enum]}_exposed.hv_{i}.json' for enum,i in  enumerate(a)]
        b = [f'model_{model_name}_{f[enum]}_naive.hv_{i}.json' for enum,i in  enumerate(b)]
    else:
        a = [f'model_{model_name}_{f[enum]}_exposed.hv_{i}.keras' for enum,i in  enumerate(a)]
        b = [f'model_{model_name}_{f[enum]}_naive.hv_{i}.keras' for enum,i in  enumerate(b)]
        
    return a+b 

model_lists = ['LM', 'rf', 'xgbr', 'NN','CNNLSTM',   'RNN' ,'CNN' , 'convLSTM'  ]
path_lists = [return_ab(m) for m in  model_lists]

import os

for paths in path_lists:
    for path in paths:
        pathc = 'model_out_back2/'+path
        pathn = 'model_out/'+path
        if os.path.exists(pathc):
            print('cp', pathc, pathn)
        else:
            print(f"Path does not exist: {path}")


cp model_out_back2/model_LM_JA_exposed.hv_43.keras model_out/model_LM_JA_exposed.hv_43.keras
cp model_out_back2/model_LM_JM_exposed.hv_43.keras model_out/model_LM_JM_exposed.hv_43.keras
cp model_out_back2/model_LM_JRF_exposed.hv_43.keras model_out/model_LM_JRF_exposed.hv_43.keras
cp model_out_back2/model_LM_JA_naive.hv_43.keras model_out/model_LM_JA_naive.hv_43.keras
cp model_out_back2/model_LM_JM_naive.hv_43.keras model_out/model_LM_JM_naive.hv_43.keras
cp model_out_back2/model_LM_JRF_naive.hv_43.keras model_out/model_LM_JRF_naive.hv_43.keras
cp model_out_back2/model_rf_JA_exposed.hv_6.pkl model_out/model_rf_JA_exposed.hv_6.pkl
cp model_out_back2/model_rf_JM_exposed.hv_12.pkl model_out/model_rf_JM_exposed.hv_12.pkl
cp model_out_back2/model_rf_JRF_exposed.hv_6.pkl model_out/model_rf_JRF_exposed.hv_6.pkl
cp model_out_back2/model_rf_JA_naive.hv_0.pkl model_out/model_rf_JA_naive.hv_0.pkl
cp model_out_back2/model_rf_JM_naive.hv_12.pkl model_out/model_rf_JM_naive.hv_12.pkl
cp model_out_back

cp model_out_back2/model_rf_JA_exposed.hv_6.pkl model_out/model_rf_JA_exposed.hv_6.pkl
cp model_out_back2/model_rf_JM_exposed.hv_12.pkl model_out/model_rf_JM_exposed.hv_12.pkl
cp model_out_back2/model_rf_JRF_exposed.hv_6.pkl model_out/model_rf_JRF_exposed.hv_6.pkl
cp model_out_back2/model_rf_JA_naive.hv_0.pkl model_out/model_rf_JA_naive.hv_0.pkl
cp model_out_back2/model_rf_JM_naive.hv_12.pkl model_out/model_rf_JM_naive.hv_12.pkl
cp model_out_back2/model_rf_JRF_naive.hv_20.pkl model_out/model_rf_JRF_naive.hv_20.pkl


In [ ]:
### transformer

# import torch
# import torch.nn as nn
# from transformers import ViTModel, ViTConfig
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# X = fm.CNN.exposed.data.i1.all
# X = X[:,:,:57].reshape(1196, 57 // 3, 3)
# y = fm.CNN.exposed.data.o1.all
# print(np.shape(X))
# print(np.shape(y))
# nom=19

# # Assuming you have your data loaded and preprocessed into numpy arrays X and y
# # X shape: (1000, 20, 3), y shape: (1000, 10)

# # Splitting the data into training, validation, and test sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

# # Normalizing the input data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train.reshape(-1, 3)).reshape(-1, nom, 3)
# X_val = scaler.transform(X_val.reshape(-1, 3)).reshape(-1, nom, 3)
# X_test = scaler.transform(X_test.reshape(-1, 3)).reshape(-1, nom, 3)

# # Convert data into PyTorch tensors
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
# X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
# y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
# X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# # Define the Vision Transformer model
# class VisionTransformerModel(nn.Module):
#     def __init__(self, input_dim, output_dim, num_layers=6, hidden_dim=768, num_heads=12):
#         super(VisionTransformerModel, self).__init__()
#         self.vit = ViTModel(ViTConfig(hidden_size=input_dim, num_hidden_layers=num_layers, num_attention_heads=num_heads, intermediate_size=hidden_dim))
#         self.fc = nn.Linear(hidden_dim, output_dim)
        
#     def forward(self, x):
#         outputs = self.vit(x)
#         last_hidden_state = outputs.last_hidden_state
#         x = self.fc(last_hidden_state[:, 0])  # Using only the first token's output for classification
#         return x

# # Instantiate the model
# model = VisionTransformerModel(input_dim=3, output_dim=10)

# # Define loss function and optimizer
# criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# # Define DataLoader for training
# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# # Training loop
# num_epochs = 10
# for epoch in range(num_epochs):
#     model.train()
#     for inputs, targets in train_loader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()
#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# # Evaluation
# model.eval()
# with torch.no_grad():
#     outputs = model(X_test_tensor)
#     test_loss = criterion(outputs, y_test_tensor)
#     print(f"Test Loss: {test_loss.item()}")


In [5]:

w1, dk1 = 1, ['LM', 'NN', 'rf', 'xgbr']
fm1 = ML_analysis('final_model_list', dk1, w1)
fmb1 = ML_analysis('final_model_list', dk1, w1, add1 = 'Braced_')
fm1 = load_model_index(fm1, fmb1, dk1)

w2, dk2 = 5,  ['CNN', 'CNNLSTM', 'convLSTM', 'RNN','convLSTM'] ## [] ##
fm2 = ML_analysis('final_model_list', dk2, w2)
fmb2 = ML_analysis('final_model_list', dk2, w2, add1 = 'Braced_')
fm2 = load_model_index(fm2, fmb2, dk2)



Loading Braced data .. for normal data use empty string as input ...
Here argument n is n+2th line in the file ... 
Loading Braced data .. for normal data use empty string as input ...
Here argument n is n+2th line in the file ... 


In [ ]:
# compute_stat([fm2.CNN, fm2.CNNLSTM, fm2.convLSTM, fm2.LSTM, fm2.VRNN, fm2.GRU ])
# compute_stat([fm1.rf, fm1.xgbr ])

# for i in np.arange(1,337,1):
#     fm1.xgbr.exposed.arg = [96, i, i]  ## JA: (96, 97, 98, 99, 144, 155, 167)
#     fm1.xgbr.naive.arg   = [i, i, i]  ## 5, 1 , 1
#     # for k in range(3):
#     #     specific(fm1.xgbr.exposed, k)
#     #     specific(fm1.xgbr.naive,   k)
#     compute_stat([fm1.xgbr])
    
#     if max(
#         fm1.xgbr.exposed.pc['JA'].mean().mean(),
#         fm1.xgbr.exposed.pc['JRF'].mean().mean(),
#         fm1.xgbr.naive.pc['JM'].mean().mean(),
#         fm1.xgbr.naive.pc['JA'].mean().mean(),
#         fm1.xgbr.naive.pc['JRF'].mean().mean()) > 0.75:
#         print('exposed',
#               fm1.xgbr.exposed.pc['JA'].mean().mean(), 
#               fm1.xgbr.exposed.pc['JM'].mean().mean(), 
#               fm1.xgbr.exposed.pc['JRF'].mean().mean(),
#               'naive',
#               fm1.xgbr.naive.pc['JA'].mean().mean(), 
#               fm1.xgbr.naive.pc['JM'].mean().mean(), 
#               fm1.xgbr.naive.pc['JRF'].mean().mean(),
#               '------------------', i, '\n\n\n\n\n')
# sys.exit()

#####################################################################    
#######        FINAL BLOCK .. only use for complete results #########
#####################################################################            
model1 = [fm1.LM, fm1.NN, fm1.rf, fm1.xgbr]  
model2 = [fm2.CNN, fm2.CNNLSTM, fm2.convLSTM, fm2.VRNN, fm2.LSTM, fm2.GRU]
# fm1 = compute_stat(model1)
# fm2 = compute_stat(model2)
# for m in model1: #+model2
#     print_stat_tables(m)
# for plot_what in ['nparams', 'pc']:
#     plot_final_results([m.exposed for m in model1+model2], 'final.exposed', plot_what)
#     plot_final_results([m.exposed_unseen for m in model1+model2], 'final.exposed_unseen', plot_what)
#     plot_final_results([m.naive for m in model1+model2], 'final.naive', plot_what)
#     if plot_what in ['pc']:
#         plot_final_results([m.naive_braced for m in model1+model2], 'final.naiveb', plot_what)

#######################

# fm = compute_stat([fm.NN])
# plot_noise_results(fm)
# print_stat_tables(fm.NN)

### XGBR
##     ##  7:0.66    ## 1:0.75
##2:0.74     ##      ## 


In [ ]:
    
# model1 = [fm1.LM, fm1.NN, fm1.rf, fm1.xgbr]  
# model2 = [fm2.CNN, fm2.CNNLSTM, fm2.convLSTM, fm2.VRNN, fm2.LSTM, fm2.GRU]
# fm1 = compute_stat(model1)
# fm2 = compute_stat(model2)

# for plot_what in ['nparams', 'pc', 'NRMSE']:
# for plot_what in ['NRMSE','RMSE']:
#     plot_final_results([m.exposed for m in model1+model2], 'final.exposed', plot_what)
#     plot_final_results([m.exposed_unseen for m in model1+model2], 'final.exposed_unseen', plot_what)
#     plot_final_results([m.naive for m in model1+model2], 'final.naive', plot_what)
#     if plot_what in ['pc','NRMSE','RMSE']:
#         plot_final_results([m.naive_braced for m in model1+model2], 'final.naiveb', plot_what)


In [ ]:
for m in model1+model2:
    combined_plot_noise(m)


In [ ]:
# for m in model1+model2:
#     print_stat_tables(m)
# fm1.LM = learning_curve(fm1.LM)

In [ ]:
# fm1.NN.lc_label = 'lc'
# learning_curve(fm1.NN)



In [ ]:
# lc = learning_curve(fm1.LM)   ## done on 23 June
# lc = learning_curve(fm1.NN)   ## done on 23 June
# lc = learning_curve(fm1.rf)   ## done on 23 June
# lc = learning_curve(fm2.CNN)  ## done on 23 June 
# lc = learning_curve(fm2.CNN)  ## done on 23 June 


In [9]:
# lc = learning_curve(fm2.CNNLSTM)  ## done on 9 March
# lc = learning_curve(fm2.LSTM)  ## done on 9 March

In [22]:
# plot_learning_curve('CNNLSTM','naive','JRF')

In [8]:
### to print the final statistics
# uuu = fm1[0]
# fff_exp = pd.concat(uuu.exposed.pc.values(),axis=1).values.flatten()
# print(np.mean(fff_exp), np.std(fff_exp))
# fff_nai = pd.concat(uuu.naive.pc.values(),axis=1).values.flatten()
# print(np.mean(fff_nai), np.std(fff_nai))

TypeError: 'ML_analysis' object is not subscriptable

In [ ]:
com

In [ ]:
from keras_flops import flops_calculation
import tensorflow as tf

# Define your model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(256,)),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(10)
])

# Calculate FLOPs
flops = flops_calculation(model, batch_size=1)
print(f"FLOPs: {flops}")

In [ ]:
model = load_model('exposed','JA','convLSTM',26)

In [ ]:
fm.xgbr.feature_l

In [ ]:
itertools